Experiment settings

In [1]:
import simpy
import numpy as np
import pandas as pd
import random
import math
RANDOM_SEED = 42
### problem paramters ####
order_time_max = 5
order_time_min = 1
food_prepare_min = 2
food_prepare_max = 6

n_kitchen = 2
n_counter = 2
n_customer = 20
SIM_TIME = 12 * 60 #12 hours

random.seed(RANDOM_SEED)

# helper functions, monitoring and pandas table


In [2]:
def round_down(n, decimals=1):
    multiplier = 10 ** decimals
    return abs(math.floor(n*multiplier + 0.5) / multiplier)

counter_total_wait_times = 0
env = simpy.Environment()
# ,"Clock Time","Interarrival Time","Service Time","Wait Time","Queue Length","Total Time In System"
column_names = ["Arrival Time","Service Start Time","Queue time","Exit Time","Total Wait Time","Total Service Time","Total Time in System"]
result_fifo = np.zeros((n_customer,len(column_names)))
result_tick = np.zeros((n_customer,len(column_names)))


# Simulation setting
## A) FIFO Algorithim

In [3]:
class Counter(object):
#     Counters to take order
    def __init__(self,env,num_counter):
        self.env = env
        self.counter = simpy.Resource(env,num_counter)
        self.counter_waiting = 0
        self.service_start = None
        
    def take_order(self,cus,env):
        self.service_start = env.now
        print("%s is placing order at counter %.2f" %(cus,self.service_start))
        time_taken_to_place_order = np.random.uniform(order_time_min, order_time_max)
        yield self.env.timeout(time_taken_to_place_order)
        print("Order of %s sent to kitchen at %.2f" %(cus, env.now))
        # yield env.process(tank_truck(env, fuel_pump))

    def receive_order(self,cus,env,resource):
        global counter_total_wait_times

        with resource.kitchen.request() as my_turn:
            yield my_turn
            yield env.process(resource.prepare_food(cus,env))
            service_end = env.now
            print("%s collected the food at %.2f" %(cus, service_end))
            # Record idle counter and add to total count
            counter_total_wait_times += (service_end-self.service_start)

class Kitchen(object):
    # Kitchen to prepare food
    def __init__(self,env,num_kitchen):
        self.env = env
        self.kitchen = simpy.Resource(env,num_kitchen)
    
    def prepare_food(self,cus,env):
        print("Kitchen is preparing food for %s at %.2f" %(cus, env.now))
        food_prepare_time = np.random.uniform(food_prepare_min,food_prepare_max)
        yield self.env.timeout(food_prepare_time)
        print("Cooked food for %s at %.2f" %(cus, env.now))

def customer(env, label, queue, kitchen, data):
#     the customer process arrive at the restaurant and request counter to take order
    label = label-1
    arrive_time = env.now
    print("%s entering the queue at %.2f"%(label,arrive_time))
#     data[label,0]=label
    data[label,0]= round_down(arrive_time)
    with queue.counter.request() as my_turn:
        yield my_turn
        service_start = env.now
        data[label,1] = round_down(service_start)
        queue_time = service_start - arrive_time
        data[label,2]= round_down(queue_time)
        # placing order at counter
        yield env.process(queue.take_order(label,env))
        # waiting order at counter
        prepare_food_start = env.now
        # counter is idle now
        yield env.process(queue.receive_order(label,env,kitchen))
        # prepare_food_end = round_down(env.now)
        # counter_total_wait_times += round_down(prepare_food_end - prepare_food_start)
        # receive food from counter
        prepare_food_end = env.now
        data[label,3] = round_down(prepare_food_end)

        kitchen_prepare_duration = (prepare_food_end-prepare_food_start)
        # total wait time
        data[label,4] = round_down(kitchen_prepare_duration+queue_time)
        # total service time
        data[label,5] = round_down(prepare_food_end-service_start)
        # total time in system
        data[label,6] = round_down(prepare_food_end-arrive_time)


## Start FIFO Simulation

In [5]:
# Simlating possion process for customer arrival
def customer_arrivals(env, num_counter,num_kitchen):
    """Create new *customer* until the sim time reaches 120. with poisson process"""
    counter = Counter(env,num_counter)
    kitchen = Kitchen(env,num_kitchen)
    for i in range(n_customer):
        yield env.timeout(random.expovariate(1 / 5))
        env.process(customer(env, i+1, counter, kitchen, result_fifo))

env.process(customer_arrivals(env, n_counter,n_kitchen))
env.run(until=SIM_TIME)

0 entering the queue at 5.10
0 is placing order at counter 5.10
1 entering the queue at 5.23
1 is placing order at counter 5.23
2 entering the queue at 6.84
Order of 1 sent to kitchen at 7.67
Kitchen is preparing food for 1 at 7.67
Order of 0 sent to kitchen at 8.05
Kitchen is preparing food for 0 at 8.05
3 entering the queue at 8.10
Cooked food for 1 at 10.52
1 collected the food at 10.52
2 is placing order at counter 10.52
Cooked food for 0 at 12.23
0 collected the food at 12.23
3 is placing order at counter 12.23
4 entering the queue at 14.77
Order of 2 sent to kitchen at 15.33
Kitchen is preparing food for 2 at 15.33
Order of 3 sent to kitchen at 16.46
Kitchen is preparing food for 3 at 16.46
Cooked food for 2 at 17.63
2 collected the food at 17.63
4 is placing order at counter 17.63
Order of 4 sent to kitchen at 19.66
Kitchen is preparing food for 4 at 19.66
Cooked food for 3 at 20.33
3 collected the food at 20.33
5 entering the queue at 20.41
5 is placing order at counter 20.41
C

## FIFO Result & Analysis

In [14]:

# Converting to pandas
np_arr = np.array(result_fifo).reshape(n_customer,-1)
labels = [*range(1,n_customer+1)]
df_fifo=pd.DataFrame(data = np_arr,index=labels,columns=column_names)

total_wait_time = np.sum(df_fifo["Total Wait Time"])
total_time_in_system = np.sum(df_fifo["Total Time in System"])
total_service_time = np.sum(df_fifo['Total Service Time'])
print("")

def display(i):
    print()
    print("Counter Total Idle Time => %.2f Minutes" % counter_total_wait_times)
    print()
    print("Average Waiting Time => %.2f Minutes" % (total_wait_time / i))
    print("Average Service Time => %.2f Minutes" % (total_service_time / i))
    print("Average Time Spent In System => %.2f Minutes" % (total_time_in_system / i))

display(n_customer)



Counter Total Idle Time => 94.05 Minutes

Average Waiting Time => 16.30 Minutes
Average Service Time => 8.75 Minutes
Average Time Spent In System => 19.21 Minutes
